# Generate daily bars using dedicated data service - Python

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.<br>

This sample demonstrates how to directly request and plot *`daily bars`*.<br>

It enables to retrieve **on-demand** daily bars data by calling a dedicated service on a `daily data store`. 

### Inputs/outputs
Bars sample requires instrument's identifier as per input. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve daily bars from the dedicated hosted service. The queried endpoint in this script are:
* *DailyBarsService*: to directly retrieve daily bars objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.type.shared.v1*
    * *systemathics.apis.services.daily.v1*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display package

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas mplfinance plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.services.daily.v1.daily_bars_pb2 as daily_bars
import systemathics.apis.services.daily.v1.daily_bars_pb2_grpc as daily_bars_service

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Create and process request

#### 3.1 Request Parameters
To request *daily bars* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Service called: *DailyBarsService* in order to retrieve daily data

In [ ]:
# set the instrument
ticker = 'AAPL'
exchange = 'XNGS'

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
# create the daily bars request
request = daily_bars.DailyBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
)

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel(os.environ("GRPC_APIS"), credentials) as channel:
    
    # instantiate the daily prices service
    service = daily_bars_service.DailyBarsServiceStub(channel)
    
    # process the request
    response = service.DailyBars(
        request = request, 
        metadata = [('authorization', token)]
    )
    
print("Total bars retrieved: ",len(response.data))

### Step 4: Visualize data

#### 4.1 Retrieve daily bars data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
#pPrepare the data frame content
dates=[datetime(b.date.year, b.date.month, b.date.day) for b in response.data]
opens = [b.open for b in response.data]
highs = [b.high for b in response.data]
lows = [b.low for b in response.data]
closes = [b.close for b in response.data]
volumes = [b.volume for b in response.data]

d = {'Date': dates, 'Open': opens, 'High': highs, 'Low' : lows,'Close': closes, 'Volume': volumes }

In [ ]:
# create pandas dataframe
df = pd.DataFrame(data=d)

In [ ]:
# set date as dataframe index
df = df.set_index('Date')

In [ ]:
# visualize dataframe
df

#### 4.2 Plot daily bars with mplfinance
The following code snippets display daily bars' request results with the package of your choice.<br>*`mplfinance`* is used as per open source display package.

In [ ]:
# set number of days
N = 100

In [ ]:
import mplfinance as mpf

mpf.plot(df.tail(N), figratio=(40,15),
         type = 'candle', 
         title=('Daily bars for {0}-{1}: last {2} days'.format(ticker,exchange,N)),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

#### 4.3 Plot daily bars with plotly
The following code snippets display daily bars' request results with the package of your choice.<br>*`plotly`* is used as per open source display package.

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = 'Daily bars for {0}-{1}'.format(ticker,exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()